In [ ]:
import json
with open('mongo.json') as m:
    data = json.load(m)
    conn_string = data["connection_string"]

In [ ]:
import requests as req
import time 
import random

import pymongo
mongo_client = pymongo.MongoClient(conn_string)

mydb = mongo_client["bigData"]